In [11]:
import torch
import torchvision
from danns_eg.data.mnist import get_sparse_permutation_invariant_fashionmnist_dataloaders
import matplotlib.pyplot as plt
import numpy as np
import danns_eg.deepdensenet as deepdensenets
import wandb
from torch.cuda.amp import GradScaler, autocast

In [13]:
# Fetch runs for a specific project
def fetch_runs(api, entity, project_name, filters, order=None):
    if order:
        runs = api.runs(f"{entity}/{project_name}", filters=filters, order=order)
    else:
        runs = api.runs(f"{entity}/{project_name}", filters=filters)
    #print(f"Runs for project '{project_name}':")
    return runs

In [14]:
class NestedObject:
    def __init__(self, params):
        for key, value in params.items():
            if isinstance(value, dict):  # If the value is a dictionary, recursively convert it
                setattr(self, key, NestedObject(value))
            else:
                setattr(self, key, value)

In [4]:
 # model.is_dann, model.homeostasis, model.normtype, model.homeo_opt_exc, opt.lambda_homeo , opt.lambda_homeo_var, opt.use_sep_bias_gain_lrs, model.implicit_homeostatic_loss


params = {
    "model": {
        "is_dann": 1,
        "homeostasis": 0,
        "normtype": 0,
        "implicit_homeostatic_loss": 0,
        "homeo_opt_exc": 0,
        "hidden_layer_width": 200
        },
    "opt": {
        "lambda_homeo": 1,
        "lambda_homeo_var": 1,
        "use_sep_bias_gain_lrs": 0
    },
}

# Create the object from the nested dictionary
p = NestedObject(params)

In [15]:
# Load FashionMNIST dataset
dataloader = get_sparse_permutation_invariant_fashionmnist_dataloaders(permutation_invariant=False, brightness_factor=0.75)


In [16]:
dataiter = iter(dataloader['train'])
images, labels = next(dataiter)

In [17]:
api = wandb.Api(timeout=19)

In [18]:
bright_factor = 0.75
runs_dict = dict()
runs_dict["runs_homeostasis"] = fetch_runs(api, entity='project_danns', project_name='Luminosity_NAISYS', filters={"config.dataset": "fashionmnist", 
                                                                "config.brightness_factor": bright_factor,"config.homeostasis": 1, "config.normtype": 0,
                                                                "config.use_testset": True, "config.task_opt_inhib": 1, "config.use_sep_bias_gain_lrs": 0,
                                                                "config.homeostatic_annealing": 0, "config.implicit_homeostatic_loss": 0 }, order="-summary_metrics.test_acc")

In [23]:
for run in runs_dict["runs_homeostasis"]:
    params = {
        "model": {
            "is_dann": run.config['is_dann'],
            "homeostasis": run.config['homeostasis'],
            "normtype": run.config['normtype'],
            "implicit_homeostatic_loss": run.config['implicit_homeostatic_loss'],
            "homeo_opt_exc": run.config['homeo_opt_exc'],
            "hidden_layer_width": run.config['hidden_layer_width']
            },
        "opt": {
            "lambda_homeo": run.config['lambda_homeo'],
            "lambda_homeo_var": run.config['lambda_homeo_var'],
            "use_sep_bias_gain_lrs": run.config['use_sep_bias_gain_lrs']
        },
    }

    # Create the object from the nested dictionary
    p = NestedObject(params)

    model = deepdensenets.net(p)
    model.load_state_dict(torch.load(f"/network/scratch/r/roy.eyono/explicit_loss_models_fashionmnist_{bright_factor}/{run.name}.pth", map_location=torch.device('cuda')))
    model.eval()
    with autocast():
        print(model(images.squeeze(1)))
    break

tensor([[2.4586e-38, 1.0538e-34, 1.0041e-37, 1.3078e-33, 2.3287e-36, 2.3307e-16,
         1.8189e-36, 9.2597e-15, 1.6557e-25, 1.0000e+00],
        [1.0000e+00, 8.7096e-29, 2.9595e-23, 1.5146e-16, 3.9849e-29, 2.1352e-29,
         2.5037e-17, 2.7164e-33, 2.3567e-29, 2.3142e-31],
        [2.6477e-03, 2.9595e-02, 1.8392e-09, 9.6775e-01, 3.7298e-06, 1.0170e-11,
         3.8157e-11, 2.7369e-09, 2.8470e-11, 6.6110e-11],
        [8.3549e-01, 1.1942e-06, 1.9691e-09, 1.6451e-01, 2.1250e-10, 7.2762e-13,
         3.7035e-10, 1.1286e-12, 4.3479e-13, 7.0897e-13],
        [4.2860e-07, 1.9138e-03, 1.5960e-12, 9.9809e-01, 3.8645e-08, 1.1030e-11,
         3.1403e-14, 7.5824e-08, 1.5493e-10, 2.2912e-11],
        [3.5082e-18, 6.8038e-20, 1.0000e+00, 1.0857e-16, 1.8442e-10, 2.8431e-18,
         3.2379e-14, 5.1212e-23, 4.1910e-19, 3.6390e-23],
        [1.6312e-08, 4.6282e-07, 1.8111e-06, 1.4855e-06, 4.7930e-07, 1.5839e-01,
         2.7090e-07, 8.4156e-01, 4.3290e-05, 6.4466e-06],
        [3.0288e-13, 1.5661

In [5]:
 # model.is_dann, model.homeostasis, model.normtype, model.homeo_opt_exc, opt.lambda_homeo , opt.lambda_homeo_var, opt.use_sep_bias_gain_lrs, model.implicit_homeostatic_loss
model = deepdensenets.net(p)

In [ ]:
model(images)